## DPDS Test Script
This script will run a benchmark upon a given dataset, default dataset download: http://boston.lti.cs.cmu.edu/appendices/WWW2016/
Run all cells below to init DB, and run script `S3` to establish knowledge graph.
To change your data source, check `S1`, to change knowledge graph , check `S3`. To managem files, check `S4`.

In [1]:
from neo4j import GraphDatabase, basic_auth
from neo4j.exceptions import Neo4jError
import neo4j.time
import csv
import json
import time
import os
DATABASE_USERNAME="neo4j"
DATABASE_PASSWORD="spade-discounts-switch"
DATABASE_URL="bolt://localhost:7687"
def cprint(content,module='DEBUG',*args):
    if args:
        print('\033[1;32;43m ['+module+'] \033[0m '+ content + '\033[1;35m' +str(args) +' \033[0m' + time.strftime(" |%Y-%m-%d %H:%M:%S|", time.localtime()) )
    else:
        print('\033[1;32;43m ['+module+'] \033[0m '+ content + time.strftime(" |%Y-%m-%d %H:%M:%S|", time.localtime()))
        
driver = GraphDatabase.driver(DATABASE_URL, auth=basic_auth(DATABASE_USERNAME, str(DATABASE_PASSWORD)))


In [2]:
s2_doc = []
with open("./S2/s2_doc.json", "r") as f_r:
    s2_doc = f_r.readlines()
    

        

In [3]:
hasKey = 0
notHasKey = 0
for i in s2_doc:
    row = json.loads(i[:-1])
    if not 'keyPhrases' in row:
        notHasKey +=1
    else:
        hasKey +=1
        

In [4]:
print(len(s2_doc),hasKey,notHasKey)

8541 6949 1592


In [5]:
article_props = []
tag_props = []
for i in s2_doc:
    row = json.loads(i[:-1])
    if 'keyPhrases' in row:
        article_prop = {
            'title':row['title'][0],
            'keyPhrases':'+'.join(row['keyPhrases']),
            'docno':row['docno'],
            'paperAbstract':row['paperAbstract'][0],
            'numCitedBy':row['numCitedBy'][0],
            'numKeyCitations':row['numKeyCitations'][0]
        }
        if not article_prop in article_props:
            article_props.append(article_prop)

        for tag in row['keyPhrases']:
            tag_prop = { 'name': tag }
            if not tag_prop in tag_props:
                tag_props.append(tag_prop)

In [6]:
len(article_props)

6949

In [7]:
len(tag_props)

18582

In [8]:
def db_create_nodes(session,labels,properties):
    '''
    Create nodes with custom labels and properties
    Args:
        session: db session,driver.session()
        labels: ["Human", "MovieStar"]
        properties => [{name: "Tom Cruise", placeOfBirth: "Syracuse, New York, United States"},
                        {name: "Reese Witherspoon", placeOfBirth: "New Orleans, Louisiana, United States"}]
    Return:
        Added nodes 
    '''
    def _cypher(tx, labels, properties):
                return list(tx.run(
                    '''
                    CALL apoc.create.nodes( $labels, $properties);
                    ''', {'labels': labels , 'properties': properties}
                ))
    result = session.write_transaction(_cypher,labels,properties)
    cprint(str(len(result))+'record added','DB')
    return result
def db_create_tag_relation_to_article(session,tag_name,article_url):
    '''
    Args: db session, Tag.name, Article.url
    Return: cypher result of created relation
    '''
    def _cypher(tx,tag_name,article_url):
#CALL apoc.create.relationship(a, "HAS_TAG",{article_id:$article_id, tag_name:$tag_name},  t)
        return list(tx.run(
        '''
        MATCH (a:Article {docno: $article_url})
        MATCH (b:Tag {name: $tag_name})
        MERGE (a)-[rel:HAS_TAG]->(b)
        RETURN rel
        ''',{'article_url':article_url,'tag_name':tag_name}
        
        ))    
    result = session.write_transaction(_cypher,tag_name,article_url)
    return result

In [9]:
with driver.session() as session:
    db_create_nodes(session,['Article'],article_props)
    db_create_nodes(session,['Tag'],tag_props)
    session.close()

 [DB]  6949record added |2021-05-24 22:42:15|
 [DB]  18582record added |2021-05-24 22:42:17|


In [10]:
with driver.session() as session:
    counter = 0
    for article in article_props:
        for tag in article['keyPhrases'].split('+'):
            res = db_create_tag_relation_to_article(session,tag,article['docno'])
            if res:
                counter+=1
    print(counter,'relationship added')
    session.close()

33778 relationship added


In [ ]:
#To enable Knowledge Graph, run script S3.